In [1]:
import os

In [2]:
%pwd

'/home/ryle/Desktop/End-to-end-ML-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/ryle/Desktop/End-to-end-ML-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded! with following info: \n{headers}")
            except Exception as e:
                logger.error(f"Failed to download file: {e}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile as e:
            logger.error(f"Failed to extract zip file: {e}")

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-02 00:09:40,886: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-02 00:09:40,888: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-02 00:09:40,890: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-02 00:09:40,892: INFO: common: created directory at: artifacts]
[2024-04-02 00:09:40,892: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-02 00:09:42,941: INFO: 2934765416: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 21271
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4b625c2ad76852a6a8d4c73c7531633b0d4c9c15229582f9336b4b36eb3d5b9c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E4DC:15EE52:11AD3F:170E02:660AF240
Accept-Ranges: bytes
Date: Mon, 